In [1]:
# For character vectorization, it is an extremely simple process of just mapping each
# character of the term to a corresponding unique number
import numpy as np
import pandas as pd

In [2]:
# This function takes a list of words or terms and returns the corresponding character
# vectors for the words.
def vectorize_terms(terms):
    terms = [term.lower() for term in terms]
    terms = [np.array(list(term)) for term in terms]
    terms = [np.array([ord(char) for char in term]) 
                for term in terms]
    return terms

In [6]:
root = 'όμορφος'
term1 = 'όμοφρος'
term2 = 'όμορρος'
term3 = 'καλύτερα'    

terms = [root, term1, term2, term3]
terms

['όμορφος', 'όμοφρος', 'όμορρος', 'καλύτερα']

In [7]:
# Let's now perform character vectorization on each of these strings (list of character
# tokens) and view their representation in the form of a data frame.
# Character vectorization
term_vectors = vectorize_terms(terms)

# show vector representations
vec_df = pd.DataFrame(term_vectors, index=terms)
print(vec_df)

            0    1    2    3    4    5    6      7
όμορφος   972  956  959  961  966  959  962    NaN
όμοφρος   972  956  959  966  961  959  962    NaN
όμορρος   972  956  959  961  961  959  962    NaN
καλύτερα  954  945  955  973  964  949  961  945.0


Thus you can see how we can easily transform each text term into a corresponding numeric vector representation.

Several distance metrics:
• Hamming distance
• Manhattan distance
• Euclidean distance
• Levenshtein Edit distance
• Cosine distance and similarity

Use the power of NumPy arrays to implement the necessary computations and mathematical formulae.

In [8]:
root_term = root
other_terms = [term1, term2, term3]

root_term_vec = vec_df[vec_df.index == root_term].dropna(axis=1).values[0]
other_term_vecs = [vec_df[vec_df.index == term].dropna(axis=1).values[0]
                      for term in other_terms]

### Hamming Distance (normalized)
defined as the number of positions that have different characters or symbols between two strings

In [9]:
def hamming_distance(u, v, norm=False):
    if u.shape != v.shape:
        raise ValueError('The vectors must have equal lengths.')
    return (u != v).sum() if not norm else (u != v).mean()

In [10]:
# We can measure the Hamming distance between our root term and the other terms
# using the following code snippet.
# the smaller the score more, the similar the terms

for term, term_vector in zip(other_terms, other_term_vecs):
    print('Hamming distance between root: {} and term: {} is {}'.format(root_term,
                                                                        term,
                                                                        hamming_distance(root_term_vec, 
                                                                                         term_vector, norm=False)))

Hamming distance between root: όμορφος and term: όμοφρος is 2
Hamming distance between root: όμορφος and term: όμορρος is 1


ValueError: The vectors must have equal lengths.

In [11]:
for term, term_vector in zip(other_terms, other_term_vecs):
    print('Normalized Hamming distance between root: {} and term: {} is {}'.format(root_term,
                                                                                   term,
                                                                                   round(hamming_distance(root_term_vec, 
                                                                                         term_vector, norm=True),
                                                                                         2)
                                                                                   ))

Normalized Hamming distance between root: όμορφος and term: όμοφρος is 0.29
Normalized Hamming distance between root: όμορφος and term: όμορρος is 0.14


ValueError: The vectors must have equal lengths.

### Manhattan Distance

In [12]:
def manhattan_distance(u, v, norm=False):
    if u.shape != v.shape:
        raise ValueError('The vectors must have equal lengths.')
    return abs(u - v).sum() if not norm else abs(u - v).mean()

In [13]:
for term, term_vector in zip(other_terms, other_term_vecs):
    print('Manhattan distance between root: {} and term: {} is {}'.format(root_term,
                                                                          term,
                                                                          manhattan_distance(root_term_vec, 
                                                                                             term_vector, norm=False)))

Manhattan distance between root: όμορφος and term: όμοφρος is 10
Manhattan distance between root: όμορφος and term: όμορρος is 5


ValueError: The vectors must have equal lengths.

In [14]:
for term, term_vector in zip(other_terms, other_term_vecs):
    print('Normalized Manhattan distance between root: {} and term: {} is {}'.format(root_term,
                                                                                     term,
                                                                                     round(manhattan_distance(root_term_vec, 
                                                                                           term_vector, norm=True),
                                                                                           2)
                                                                                     ))

Normalized Manhattan distance between root: όμορφος and term: όμοφρος is 1.43
Normalized Manhattan distance between root: όμορφος and term: όμορρος is 0.71


ValueError: The vectors must have equal lengths.

### Euclidean Distance

In [15]:
def euclidean_distance(u,v):
    if u.shape != v.shape:
        raise ValueError('The vectors must have equal lengths.')
    distance = np.sqrt(np.sum(np.square(u - v)))
    return distance

In [16]:
for term, term_vector in zip(other_terms, other_term_vecs):
    print('Euclidean distance between root: {} and term: {} is {}'.format(root_term,
                                                                          term,
                                                                          round(euclidean_distance(root_term_vec, 
                                                                                                   term_vector),
                                                                                2)
                                                                          ))

Euclidean distance between root: όμορφος and term: όμοφρος is 7.07
Euclidean distance between root: όμορφος and term: όμορρος is 5.0


ValueError: The vectors must have equal lengths.

### Levenshtein_edit_distance

In [17]:
import copy
import pandas as pd

def levenshtein_edit_distance(u, v):
    # convert to lower case
    u = u.lower()
    v = v.lower()
    # base cases
    if u == v: return 0
    elif len(u) == 0: return len(v)
    elif len(v) == 0: return len(u)
    # initialize edit distance matrix
    edit_matrix = []
    # initialize two distance matrices 
    du = [0] * (len(v) + 1)
    dv = [0] * (len(v) + 1)
    # du: the previous row of edit distances
    for i in range(len(du)):
        du[i] = i
    # dv : the current row of edit distances    
    for i in range(len(u)):
        dv[0] = i + 1
        # compute cost as per algorithm
        for j in range(len(v)):
            cost = 0 if u[i] == v[j] else 1
            dv[j + 1] = min(dv[j] + 1, du[j + 1] + 1, du[j] + cost)
        # assign dv to du for next iteration
        for j in range(len(du)):
            du[j] = dv[j]
        # copy dv to the edit matrix
        edit_matrix.append(copy.copy(dv))
    # compute the final edit distance and edit matrix    
    distance = dv[len(v)]
    edit_matrix = np.array(edit_matrix)
    edit_matrix = edit_matrix.T
    edit_matrix = edit_matrix[1:,]
    edit_matrix = pd.DataFrame(data=edit_matrix,
                               index=list(v),
                               columns=list(u))
    return distance, edit_matrix

In [18]:
for term in other_terms:
    edit_d, edit_m = levenshtein_edit_distance(root_term, term)
    print('Computing distance between root: {} and term: {}'.format(root_term,
                                                                    term))
    print('Levenshtein edit distance is {}'.format(edit_d))
    print('The complete edit distance matrix is depicted below')
    print(edit_m)
    print('-'*30)

Computing distance between root: όμορφος and term: όμοφρος
Levenshtein edit distance is 2
The complete edit distance matrix is depicted below
   ό  μ  ο  ρ  φ  ο  ς
ό  0  1  2  3  4  5  6
μ  1  0  1  2  3  4  5
ο  2  1  0  1  2  3  4
φ  3  2  1  1  1  2  3
ρ  4  3  2  1  2  2  3
ο  5  4  3  2  2  2  3
ς  6  5  4  3  3  3  2
------------------------------
Computing distance between root: όμορφος and term: όμορρος
Levenshtein edit distance is 1
The complete edit distance matrix is depicted below
   ό  μ  ο  ρ  φ  ο  ς
ό  0  1  2  3  4  5  6
μ  1  0  1  2  3  4  5
ο  2  1  0  1  2  3  4
ρ  3  2  1  0  1  2  3
ρ  4  3  2  1  1  2  3
ο  5  4  3  2  2  1  2
ς  6  5  4  3  3  2  1
------------------------------
Computing distance between root: όμορφος and term: καλύτερα
Levenshtein edit distance is 8
The complete edit distance matrix is depicted below
   ό  μ  ο  ρ  φ  ο  ς
κ  1  2  3  4  5  6  7
α  2  2  3  4  5  6  7
λ  3  3  3  4  5  6  7
ύ  4  4  4  4  5  6  7
τ  5  5  5  5  5  6  7
ε  6 

### Cosine Distance and Similarity
Bag of characters vectorization is very similar to the bag of words model except here we compute the frequency of each character in the word. Sequence or word orders are not taken into account here.

In [19]:
def boc_term_vectors(word_list):
    word_list = [word.lower() for word in word_list]
    unique_chars = np.unique(
                        np.hstack([list(word) 
                        for word in word_list]))
    word_list_term_counts = [{char: count 
                                  for char, count in np.stack(
                                                         np.unique(list(word), 
                                                                   return_counts=True),
                                                         axis=1)}
                                 for word in word_list]
    
    boc_vectors = [np.array([int(word_term_counts.get(char, 0)) 
                            for char in unique_chars])
                   for word_term_counts in word_list_term_counts]
    return list(unique_chars), boc_vectors

In [20]:
# Bag of characters vectorization
import pandas as pd

feature_names, feature_vectors = boc_term_vectors(terms)
boc_df = pd.DataFrame(feature_vectors, columns=feature_names, index=terms)
print(boc_df)

          α  ε  κ  λ  μ  ο  ρ  ς  τ  φ  ό  ύ
όμορφος   0  0  0  0  1  2  1  1  0  1  1  0
όμοφρος   0  0  0  0  1  2  1  1  0  1  1  0
όμορρος   0  0  0  0  1  2  2  1  0  0  1  0
καλύτερα  2  1  1  1  0  0  1  0  1  0  0  1


In [21]:
root_term_boc = boc_df[vec_df.index == root_term].values[0]
other_term_bocs = [boc_df[vec_df.index == term].values[0]
                      for term in other_terms]

In [22]:
def cosine_distance(u, v):
    distance = 1.0 - (np.dot(u, v) / 
                        (np.sqrt(sum(np.square(u))) * np.sqrt(sum(np.square(v))))
                     )
    return distance

In [23]:
for term, boc_term in zip(other_terms, other_term_bocs):
    print('Analyzing similarity between root: {} and term: {}'.format(root_term,
                                                                      term))
    distance = round(cosine_distance(root_term_boc, boc_term), 2)
    similarity = round(1 - distance, 2)                                                           
    print('Cosine distance  is {}'.format(distance))
    print('Cosine similarity  is {}'.format(similarity))
    print('-'*40)

Analyzing similarity between root: όμορφος and term: όμοφρος
Cosine distance  is 0.0
Cosine similarity  is 1.0
----------------------------------------
Analyzing similarity between root: όμορφος and term: όμορρος
Cosine distance  is 0.1
Cosine similarity  is 0.9
----------------------------------------
Analyzing similarity between root: όμορφος and term: καλύτερα
Cosine distance  is 0.89
Cosine similarity  is 0.11
----------------------------------------


These vector representations do not take the order of characters into account
and hence the similarity between the terms "Believe" and "beleive" is 1.0 or a perfect 100%. 

You can see how this can be used in combination with a semantic dictionary like WordNet to provide correct spelling suggestions. It does this by suggesting semantically and syntactically correct words from a vocabulary when users misspell a word by measuring the similarity between the words!!!